In [ ]:
%run ../ssb_sparktools/processing/processing.py

In [ ]:
skatt_raw = spark.read.path("/kilde/ske/skatt/person/fastsatt/rådata/2019v1.0.1-20200702")

In [ ]:
skattemelding_hendelse = skatt_raw\
                        .select('hendelse.*', 'skattemeldingUtflatet')\
                        .withColumnRenamed('registreringstidspunkt', 'hendelsetidspunkt')\
                        .withColumn('hendelsetidspunkt', F.to_timestamp('hendelsetidspunkt', "yyyy-MM-dd'T'HH:mm:ss.SSS'Z'"))
skattemelding_utflatet = skattemelding_hendelse\
                        .select('sekvensnummer','identifikator', 'gjelderPeriode', 'hendelsetidspunkt', 'hendelsetype', 'skattemeldingUtflatet.*')\
                        .withColumn('registreringstidspunkt', F.to_timestamp('registreringstidspunkt', "yyyy-MM-dd'T'HH:mm:ss.SSS'Z'"))\
                        .withColumn('hendelsetype', F.upper(F.col('hendelsetype')))

dupvars = list(skattemelding_utflatet.columns)
dupvars.remove('sekvensnummer')

skattemelding_nodup = skattemelding_utflatet.dropDuplicates(subset=dupvars) 
                    
skattemelding_tverrsnitt =  cross_sectional(skattemelding_nodup, 'hendelsetidspunkt', ['identifikator', 'gjelderPeriode']).cache()

tverrsnitt_filtrert = skattemelding_tverrsnitt\
                        .filter(F.col('hendelsetype')!='SLETTET')\
                        .filter(F.col('skjermet')==False)\
                        .filter(F.col('identifikator')==F.col('personidentifikator'))

In [ ]:
skattemelding_tverrsnitt.count()

In [ ]:
testdf = 'ikkedf'
skattemelding_tverrsnitt_feildf = cross_sectional(testdf, 'hendelsetidspunkt', ['identifikator', 'gjelderPeriode'])

In [ ]:
testvar = 0
skattemelding_tverrsnitt_mcutoff = cross_sectional(skattemelding_nodup, testvar, ['identifikator', 'gjelderPeriode'], None)

In [ ]:
skattemelding_tverrsnitt_mcutoff = cross_sectional(skattemelding_nodup, 'hendelsetidspunkt', 'ikkeliste', None)

In [ ]:
import datetime
cotime = datetime.datetime.strptime('2020-06-01', '%Y-%m-%d')
skattemelding_tverrsnitt_mcutoff = cross_sectional(skattemelding_nodup, 'hendelsetidspunkt', ['identifikator', 'gjelderPeriode'], cotime)
skattemelding_tverrsnitt_mcutoff.count()

In [ ]:
skattemelding_tverrsnitt_feilcutoff = cross_sectional(skattemelding_nodup, 'hendelsetidspunkt', ['identifikator', 'gjelderPeriode'], '2020-06-01')

In [ ]:
so_dict_testtverrsnitt = unpack_parquet(tverrsnitt_filtrert, rootdf=True, rootvar=['personidentifikator'])
for k in so_dict_testtverrsnitt.keys():
    print(k)

In [ ]:
so_dict_testtverrsnitt['konto'].toPandas()